<a href="https://colab.research.google.com/github/Agod1/demos/blob/master/CSRNet_with_MobileNet_People_Counting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Conv2D, Reshape, Input
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import scipy.io
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

# Define input and output dimensions
IN_X, IN_Y = 768, 1024
OUT_X, OUT_Y = 96, 128
LR = 1e-4  # Learning rate
BATCH_SIZE = 1
EPOCHS = 1000
checkpoint_filepath = '/content/drive/MyDrive/Teaching/TME_6015/ShanghaiTech_Crowd_Counting_Dataset/part_A_final/test_data/images/my_checkpoint.weights.h5'

In [ ]:

def get_base_model(input_shape=(IN_X, IN_Y, 3)):
    # Load VGG-16 up to block4_conv3 layer for feature extraction
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    output = base_model.get_layer("block4_conv3").output
    return Model(inputs=base_model.input, outputs=output)

# CSRNet Architecture with dilated convolutions for the back-end
inputs = Input(shape=(IN_X, IN_Y, 3))
x = get_base_model()(inputs)
init = RandomNormal(stddev=0.01)

# Add dilated convolutions for the back-end
x = Conv2D(512, (3, 3), activation='relu', dilation_rate=2, kernel_initializer=init, padding='same')(x)
x = Conv2D(512, (3, 3), activation='relu', dilation_rate=2, kernel_initializer=init, padding='same')(x)
x = Conv2D(512, (3, 3), activation='relu', dilation_rate=2, kernel_initializer=init, padding='same')(x)
x = Conv2D(256, (3, 3), activation='relu', dilation_rate=2, kernel_initializer=init, padding='same')(x)
x = Conv2D(128, (3, 3), activation='relu', dilation_rate=2, kernel_initializer=init, padding='same')(x)
x = Conv2D(64, (3, 3), activation='relu', dilation_rate=2, kernel_initializer=init, padding='same')(x)
x = Conv2D(1, (1, 1), activation='relu', kernel_initializer=init, padding='same')(x)

# Reshape output for density map
out = Reshape((OUT_X, OUT_Y))(x)
model = Model(inputs=inputs, outputs=out)
model.summary()


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 768, 1024, 3)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ functional (Functional)              │ (None, 96, 128, 512)        │       7,635,264 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 96, 128, 512)        │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 96, 128, 512)        │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 96, 128, 512)        │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 96, 128, 256)        │       1,179,904 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 96, 128, 128)        │         295,040 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 96, 128, 64)         │          73,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_6 (Conv2D)                    │ (None, 96, 128, 1)          │              65 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape (Reshape)                    │ (None, 96, 128)             │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 16,263,489 (62.04 MB)

 Trainable params: 16,263,489 (62.04 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:

# Compile the model
model.compile(
    loss='mean_squared_error',
    optimizer=Adam(learning_rate=LR),
    metrics=['mae'],
    run_eagerly=True
)


In [ ]:
def gauss_distribution(x,u=0,sigma=10):
    return np.expand_dims(1/(np.sqrt(2*np.pi*(sigma**2)))*np.exp(-(0.5)*(((x-u)/sigma)**2)),axis=0)

In [ ]:
def get_density_map_gaussian(im, points, gaussian_radius=4): # Updated with sigma as input
    """
    Generates a density map using Gaussian kernels centered at given points.

    Args:
        im (numpy.ndarray): The input image.
        points (list): A list of (x, y) coordinates representing the points.
        gaussian_radius (int, optional): The radius of the Gaussian kernel. Defaults to 4.

    Returns:
        numpy.ndarray: The generated density map.
    """
    density_map = np.zeros((OUT_X, OUT_Y))
    w, h = OUT_Y, OUT_X
    num_gt = len(points)

    for point in points:
        point = np.round(point).astype(int)
        # Check if point coordinates are within image boundaries
        if 0 <= point[1] < h and 0 <= point[0] < w:
            point[0], point[1] = min(h - 1, point[1]), min(w - 1, point[0])
            x = np.linspace(-gaussian_radius, gaussian_radius, (gaussian_radius * 2) + 1)
            gaussian_map = np.multiply(gauss_distribution(x), gauss_distribution(x).T)
            gaussian_map /= np.sum(gaussian_map)

            x_left, x_right, y_up, y_down = 0, gaussian_map.shape[1], 0, gaussian_map.shape[0]
            if point[1] < gaussian_radius:
                x_left = gaussian_radius - point[1]
            if point[0] < gaussian_radius:
                y_up = gaussian_radius - point[0]
            if point[1] + gaussian_radius >= w:
                x_right = gaussian_map.shape[1] - (gaussian_radius + point[1] - w) - 1
            if point[0] + gaussian_radius >= h:
                y_down = gaussian_map.shape[0] - (gaussian_radius + point[0] - h) - 1

                # Check if the slice is valid
                if y_down - y_up > 0 and x_right - x_left > 0:
                    density_map[
                        max(0, point[0] - gaussian_radius):min(density_map.shape[0], point[0] + gaussian_radius + 1),
                        max(0, point[1] - gaussian_radius):min(density_map.shape[1], point[1] + gaussian_radius + 1),
                    ] += gaussian_map[y_up:y_down, x_left:x_right]
    density_map /= np.sum(density_map / len(points))
    return density_map

In [ ]:
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, images, maps, batch_size, SUBSAMPLING_FACTOR=8):

        self.images = images
        self.maps = maps
        self.batch_size = batch_size
        self.train_image_list = os.listdir(images)
        self.SUBSAMPLING_FACTOR = SUBSAMPLING_FACTOR

    def __len__(self):
        return int(np.floor(len(self.train_image_list) / self.batch_size))

    def __getitem__(self, idx):
        x, y = self.__data_generation(idx)
        return x, y

    def __data_generation(self, idx):
        x = []
        y = []

        for j in range(idx * self.batch_size, (idx + 1) * self.batch_size):

            im_array = img_to_array(load_img(self.images + os.listdir(self.images)[j], target_size=(IN_X, IN_Y)))
            im_array /= 255.
            im_array[:,:,0]=(im_array[:,:,0]-np.mean(im_array[:,:,0]))/np.std(im_array[:,:,0])
            im_array[:,:,1]=(im_array[:,:,1]-np.mean(im_array[:,:,1]))/np.std(im_array[:,:,1])
            im_array[:,:,2]=(im_array[:,:,2]-np.mean(im_array[:,:,2]))/np.std(im_array[:,:,2])
            x.append(im_array)
            mat=scipy.io.loadmat(self.maps+os.listdir(self.maps)[j])
            points=mat['image_info'][0][0][0][0][0]
            points/=self.SUBSAMPLING_FACTOR

            density_map_present=get_density_map_gaussian(im_array,points,gaussian_radius=5)
            y.append(density_map_present)
        return tf.convert_to_tensor(x),tf.convert_to_tensor(y)

In [ ]:
train_images= '/content/drive/MyDrive/ColabNotebooks/TME6015/ShanghaiTech_Crowd_Counting_Dataset/part_A_final/train_data/images/'
train_maps= '/content/drive/MyDrive/ColabNotebooks/TME6015/ShanghaiTech_Crowd_Counting_Dataset/part_A_final/train_data/ground_truth/'
val_images= '/content/drive/MyDrive/ColabNotebooks/TME6015/ShanghaiTech_Crowd_Counting_Dataset/part_A_final/test_data/images/'
val_maps= '/content/drive/MyDrive/ColabNotebooks/TME6015/ShanghaiTech_Crowd_Counting_Dataset/part_A_final/test_data/ground_truth/'

In [ ]:
train_gen = DataGenerator(train_images, train_maps,BATCH_SIZE)
# val_gen = DataGenerator(val_images, val_maps,BATCH_SIZE)

In [ ]:

from tensorflow.keras.callbacks import ModelCheckpoint

# Checkpoint settings
callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='loss',
    mode='min',
    save_best_only=True
)

# Start training
history = model.fit(
    train_gen,
    verbose=1,
    shuffle=True,
    epochs=EPOCHS,
    callbacks=[callback]
)


<ipython-input-42-dd468fe9ad4c>:42: RuntimeWarning: invalid value encountered in divide
  density_map /= np.sum(density_map / len(points))


Epoch 1/1000


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


154/300 ━━━━━━━━━━━━━━━━━━━━ 1:53:04 46s/step - loss: nan - mae: nan

In [ ]:

# Define the test image path and ground truth path
IN = 'GT_IMG_1'  # Example image filename
image_path = f"/content/drive/MyDrive/ColabNotebooks/TME6015/ShanghaiTech_Crowd_Counting_Dataset/part_A_final/test_data/images/{IN}.jpg"
ground_truth_path = f"/content/drive/MyDrive/ColabNotebooks/TME6015/ShanghaiTech_Crowd_Counting_Dataset/part_A_final/test_data/ground_truth/GT_{IN}.mat"

# Load and preprocess the image
im_array = img_to_array(load_img(image_path, target_size=(IN_X, IN_Y)))
im_array /= 255.0
for i in range(3):
    im_array[:, :, i] = (im_array[:, :, i] - np.mean(im_array[:, :, i])) / np.std(im_array[:, :, i])

plt.figure(figsize=(10, 8))
plt.imshow(im_array)

# Prediction
output = model.predict(np.expand_dims(im_array, axis=0))
output = np.reshape(output, (OUT_X, OUT_Y))

# Display prediction
predicted_count = np.sum(output)

# Load actual count for comparison
mat = scipy.io.loadmat(ground_truth_path)
points = mat['image_info'][0][0][0][0][0]
actual_count = len(points)

# Output Results
print(f"The actual number of people is: {actual_count}")
print(f"The predicted number of people is: {predicted_count}")

# Show density map
plt.figure(figsize=(10, 8))
plt.imshow(output, cmap='hot')
plt.colorbar()
plt.show()
